In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install -q langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import os

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = 'RAG'

In [ ]:
os.environ['GOOGLE_API_KEY'] = ''

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/embedding-001",
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", convert_system_message_to_human = True)

In [ ]:
print(model.invoke("Hi").content)

Hi there! How can I help you today?


In [ ]:
import bs4
from langchain import hub

In [ ]:
from langchain.chains import create_retrieval_chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
from langchain_chroma import Chroma

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
loader = WebBaseLoader(
    web_paths= ("https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",),
    bs_kwargs = dict(parse_only = bs4.SoupStrainer(class_ =("post-content","post-title", "post-header"))),
)

In [ ]:
doc = loader.load()

In [ ]:
doc

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='\n\n      Adversarial Attacks on LLMs\n    \nDate: October 25, 2023  |  Estimated Reading Time: 33 min  |  Author: Lilian Weng\n\n\nThe use of large language models in the real world has strongly accelerated by the launch of ChatGPT. We (including my team at OpenAI, shoutout to them) have invested a lot of effort to build default safe behavior into the model during the alignment process (e.g. via RLHF). However, adversarial attacks or jailbreak prompts could potentially trigger the model to output something undesired.\nA large body of ground work on adversarial attacks is on images, and differently it operates in the continuous, high-dimensional space. Attacks for discrete data like text have been considered to be a lot more challenging, due to lack of direct gradient signals. My past post on Controllable Text Generation is quite relevant to this topic, as attacking LLMs is ess

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
)

In [ ]:
splits = text_splitter.split_documents(doc)

In [ ]:
splits

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='Adversarial Attacks on LLMs\n    \nDate: October 25, 2023  |  Estimated Reading Time: 33 min  |  Author: Lilian Weng'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='The use of large language models in the real world has strongly accelerated by the launch of ChatGPT. We (including my team at OpenAI, shoutout to them) have invested a lot of effort to build default safe behavior into the model during the alignment process (e.g. via RLHF). However, adversarial attacks or jailbreak prompts could potentially trigger the model to output something undesired.\nA large body of ground work on adversarial attacks is on images, and differently it operates in the continuous, high-dimensional space. Attacks for discrete data like text have been considered to be a lot more challenging, due to lack of direct gradient signals. My p

In [ ]:
vectorstore = Chroma.from_documents(
    documents = splits,
    embedding = gemini_embeddings,
)


In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7aadc3603d10>, search_kwargs={})

In [ ]:
system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise. "
    "\n\n"
    "{context}"
)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human", "{input}"),
    ]
  )

In [ ]:
question_answering_chain = create_stuff_documents_chain(model, chat_prompt)

In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [ ]:
rag_chain.invoke({"input" : "What is UAT-LM?" })

{'input': 'What is UAT-LM?',
 'context': [Document(id='4a896ba6-f118-4ab0-b40d-c9d0a43510ee', metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='White-box attacks often lead to nonsensical adversarial prompts and thus they can be detected by examining perplexity. Of course, a white-box attack can directly bypass this by explicitly optimizing for lower perplexity, such as UAT-LM, a variation of UAT. However, there is a tradeoff and it can lead to lower attack success rate.'),
  Document(id='db13b5bd-2e8d-44d3-9ecb-c01d40853dd0', metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='Illustration of how Universal Adversarial Triggers (UAT) works. (Image source: Wallace et al. 2019)\n\nThe design of the loss $\\mathcal{L}_\\text{adv}$  for UAT is task-specific. Classification or reading comprehension relies on cross entropy. In their experiment, conditional text generation is configured to maximi